In [ ]:
pip install opencv-contrib-python

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def show(*img):
    
    if len(img)<2:
        show_one(img[0])
        return
              
    fig, axes = plt.subplots(1, len(img))
    
    for img_i, i in zip(img, range(len(img))):
        axes[i].imshow(img_i, cmap='gray')
        axes[i].set_title('')

    fig.set_figwidth(20)    #  ширина и
    fig.set_figheight(10)    #  высота \"Figure\

#     plt.gray()
    plt.show()


def show_one(img, n = 10):
    fig, ax = plt.subplots()
    fig.set_figwidth(n)    #  ширина и
    fig.set_figheight(n)    #  высота \"Figure\
    plt.imshow(img, cmap='gray')
    plt.show()

In [ ]:
video_path = '90_2_3.mp4'

# LK

In [ ]:
def lucas_kanade_method(video_path):
    # Read the video
    cap = cv2.VideoCapture(video_path)
    # Parameters for ShiTomasi corner detection
    feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
    # Parameters for Lucas Kanade optical flow
    lk_params = dict(
        winSize=(15, 15),
        maxLevel=2,
        criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03),
    )
    # Create random colors
    color = np.random.randint(0, 255, (100, 3))
 # Take first frame and find corners in it
    ret, old_frame = cap.read()
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)
 # Create a mask image for drawing purposes
    mask = np.zeros_like(old_frame)
    while True:
    # Read new frame
        ret, frame = cap.read()
        if not ret:
            break
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Calculate Optical Flow
        p1, st, err = cv2.calcOpticalFlowPyrLK(
            old_gray, frame_gray, p0, None, **lk_params
        )
        # Select good points
        good_new = p1[st == 1]
        good_old = p0[st == 1]

        # Draw the tracks
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
            frame = cv2.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)
        # Display the demo
        img = cv2.add(frame, mask)
        show(img)

        # Update the previous frame and previous points
        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1, 1, 2)




In [ ]:
lucas_kanade_method(video_path)

# Dense methods

In [ ]:
def dense_optical_flow(method, video_path, params=[], to_gray=False):
    # Read the video and first frame
    cap = cv2.VideoCapture(video_path)
    ret, old_frame = cap.read()

    # crate HSV & make Value a constant
    hsv = np.zeros_like(old_frame)
    hsv[..., 1] = 255

    # Preprocessing for exact method
    if to_gray:
        old_frame = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

    while True:
    # Read the next frame
        ret, new_frame = cap.read()
        frame_copy = new_frame
        if not ret:
            break

        # Preprocessing for exact method
        if to_gray:
            new_frame = cv2.cvtColor(new_frame, cv2.COLOR_BGR2GRAY)

        # Calculate Optical Flow
        flow = method(old_frame, new_frame, None, *params)

        # Encoding: convert the algorithm's output into Polar coordinates
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        # Use Hue and Value to encode the Optical Flow
        hsv[..., 0] = ang * 180 / np.pi / 2
        hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)

        # Convert HSV image into BGR for demo
        bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        show(frame_copy, bgr)

        # Update the previous frame
        old_frame = new_frame


In [ ]:

method = cv2.optflow.calcOpticalFlowSparseToDense
frames = dense_optical_flow(method, video_path, to_gray=True)


In [ ]:
method = cv2.calcOpticalFlowFarneback
params = [0.5, 3, 15, 3, 5, 1.2, 0]  # default Farneback's algorithm parameters
frames = dense_optical_flow(method, video_path, params, to_gray=True)

In [ ]:
method = cv2.optflow.calcOpticalFlowDenseRLOF
frames = dense_optical_flow(method, video_path)

# Farneback

In [ ]:
vc = cv2.VideoCapture('data/90_2_3.mp4')


def preprocess_frame(frame):
    #     frame = frame[0:470, ]
    frame = cv2.GaussianBlur(frame, (11, 11), 0)

    # hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # h, s, v = cv2.split(hsv)
    #
    # v = cv2.equalizeHist(v)
    #
    # hsv = cv2.merge([h, s, v])
    # bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    return frame, gray


# Read first frame
_, first_frame = vc.read()
# Scale and resize image
frame, prev_gray = preprocess_frame(first_frame)
div = 256
prev_gray = prev_gray // div * div + div // 2


def draw_flow(img, flow, step=15):
    h, w = img.shape[:2]
    y, x = np.mgrid[step / 2:h:step, step / 2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T
    lines = np.vstack([x, y, x + fx, y + fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)
    # vis = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2.polylines(img, lines, 0, (0, 255, 0))
    for (x1, y1), (_x2, _y2) in lines:
        cv2.circle(img, (x1, y1), 1, (0, 255, 0), -1)
    return img


#while True:
for i in range(5):
    _ret, frame_source = vc.read()
    frame = frame_source.copy()
    frame, gray = preprocess_frame(frame)
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    prev_gray = gray
    show(frame, draw_flow(frame_source, flow))

# SF

In [ ]:
def draw_flow(img, flow, step=15):
    h, w = img.shape[:2]
    y, x = np.mgrid[step / 2:h:step, step / 2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T
    lines = np.vstack([x, y, x + fx, y + fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)
    cv2.polylines(img, lines, 0, (0, 255, 0))
    for (x1, y1), (x2, y2) in lines:
        cv2.circle(img, (x1, y1), 1, (0, 255, 0), -1)
    return img


# Read first frame
_, first_frame = vc.read()
# Scale and resize image
frame, prev_gray = preprocess_frame(first_frame)
for i in range(10):
    # Read a frame from video
    _, frame = vc.read()
    # Convert new frame format`s to gray scale and resize gray frame obtained
    frame, gray = preprocess_frame(frame)
    # Calculate dense optical flow by Simple Flow
    flow = cv2.optflow.calcOpticalFlowSF(prev_gray, gray, 3, 2, 4)
    # Compute the magnitude and angle of the 2D vectors
    dense_flow = draw_flow(frame, flow)
    show(dense_flow)
    # Update previous frame
    prev_gray = gray
# Frame are read by intervals of 10 millisecond. The programs breaks out of the while loop when the user presses the ‘q’ key
